In [33]:
import sys
sys.path.append('../src')
from stocks import stocks, wishlist
import pandas as pd
import yfinance as yf
import time
from curl_cffi import requests


In [5]:
session = requests.Session(impersonate="chrome")

In [6]:
# Daily Closing Price 
data = pd.DataFrame()
for ticker in stocks:
    #print(f"Downloading {ticker}...")
    temp = yf.download(ticker, session=session, period='10y')['Close']
    data[ticker] = temp
    data.to_csv('../data/close.csv')
    time.sleep(2)  # Wait 2 seconds between requests

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [34]:
# Daily Closing Price for Wishlist
wish = pd.DataFrame()
for ticker in wishlist:
    #print(f"Downloading {ticker}...")
    temp = yf.download(ticker, session=session, period='10y')['Close']
    wish[ticker] = temp
    wish.to_csv('../data/wishlist.csv')
    time.sleep(2)  # Wait 2 seconds between requests

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
# Daily Closing Price for S&P 500
ticker = "SPY"
spy = yf.download(ticker, session=session, period='10y')['Close']
spy[ticker] = temp
spy.to_csv('../data/spy.csv')

[*********************100%***********************]  1 of 1 completed


In [30]:
# Create dataframe with ticker symbols and fundamental metrics
metrics_list = []

for ticker in data.columns:
    try:
        stock = yf.Ticker(ticker, session=session)
        info = stock.info
        
        # Get the last fiscal year end date
        report_date = info.get('lastFiscalYearEnd', None)
        if report_date:
            report_date = pd.Timestamp(report_date, unit='s').date()
        
        metrics_list.append({
            'Date': report_date,
            'Ticker': ticker,
            'MarketCap': info.get('marketCap', None),
            'P/E Ratio': info.get('trailingPE', None),
            'DividendYield': info.get('dividendYield', 0),
            'Revenue Growth': info.get('revenueGrowth', None)
        })
        time.sleep(1)  # Rate limiting
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        metrics_list.append({
            'Date': None,
            'Ticker': ticker,
            'MarketCap': None,
            'P/E Ratio': None,
            'DividendYield': None,
            'Revenue Growth': None
        })

metric = pd.DataFrame(metrics_list)
metric.to_csv('../data/metric.csv')

/Users/jacksonlu/miniforge3/envs/DSCI-532_2026_35_financebros/lib/python3.14/site-packages/yfinance/scrapers/quote.py:702: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  start = pd.Timestamp.utcnow().floor("D") - datetime.timedelta(days=365 // 2)
/Users/jacksonlu/miniforge3/envs/DSCI-532_2026_35_financebros/lib/python3.14/site-packages/yfinance/scrapers/quote.py:704: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  end = pd.Timestamp.utcnow().ceil("D")


In [31]:
metric

,Date,Ticker,MarketCap,P/E Ratio,DividendYield,Revenue Growth
0,2025-09-27,AAPL,3846888226816,33.088497,0.38,0.157
1,2025-06-30,MSFT,2986626777088,25.162178,0.90,0.167
2,2025-12-31,GOOGL,3737973161984,28.558226,0.27,0.180
3,2025-12-31,AMZN,2142690213888,27.877096,0.00,0.136
4,2025-12-31,META,1643730436096,27.686834,0.31,0.238
5,2025-01-26,NVDA,4551428538368,46.158024,0.02,0.625
6,2025-12-31,TSLA,1565026811904,389.785030,0.00,-0.031
